In [ ]:
import re
import json
from pathlib import Path

# --- KLASÖR YOLLARI ---
DATA_DIR = r"C:\Users\hukus\Downloads\Code4Her\knowledgw_base"
OUT_FILE = r"C:\Users\hukus\Downloads\Code4Her\knowledge_dataset.jsonl"

# --- VERİ AYIKLAMA FONKSİYONU ---
def parse_file(file_path: Path):
    text = file_path.read_text(encoding="utf-8")

    # "Soru:" ve "## Soru:" her iki formatı da destekleyen regex
    pattern = re.compile(
        r"(?:##\s*)?Soru[:\-–]\s*(.*?)\s*Cevap[:\-–]\s*(.*?)(?=\n\Soru[:\-–]|\n##\s*Soru[:\-–]|\Z)",
        re.DOTALL | re.IGNORECASE
    )

    qa_pairs = []
    for match in pattern.finditer(text):
        question = match.group(1).strip().replace("\n", " ")
        answer = match.group(2).strip().replace("\n", " ")
        qa_pairs.append({
            "source": file_path.name,
            "question": question,
            "answer": answer
        })
    return qa_pairs

# --- ANA FONKSİYON ---
def main():
    out_path = Path(OUT_FILE)
    data_dir = Path(DATA_DIR)

    all_data = []
    print(f"Veri klasörü: {data_dir}")
    print(f"Çıktı dosyası: {out_path}")

    for p in data_dir.glob("*.txt"):
        parsed = parse_file(p)
        all_data.extend(parsed)
        print(f"{p.name} dosyasından {len(parsed)} kayıt çıkarıldı.")

    # JSONL yaz
    with out_path.open("w", encoding="utf-8") as fout:
        for item in all_data:
            fout.write(json.dumps(item, ensure_ascii=False) + "\n")

    print(f"\nToplam {len(all_data)} kayıt {out_path} dosyasına yazıldı.")

# --- ÇALIŞTIR ---
if __name__ == "__main__":
    main()


Veri klasörü: C:\Users\hukus\Downloads\Code4Her\knowledgw_base
Çıktı dosyası: C:\Users\hukus\Downloads\Code4Her\knowledge_dataset.jsonl

Toplam 0 kayıt C:\Users\hukus\Downloads\Code4Her\knowledge_dataset.jsonl dosyasına yazıldı.


In [ ]:
import json
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# Modeli yükle
model = SentenceTransformer("intfloat/multilingual-e5-base")

# Veri dosyasının yolu
input_file = r"C:\Users\hukus\Downloads\Code4Her\knowledge_dataset.jsonl"
output_embeddings = r"C:\Users\hukus\Downloads\Code4Her\knowledge_embeddings.npy"
output_texts = r"C:\Users\hukus\Downloads\Code4Her\knowledge_texts.json"

texts = []

# JSONL dosyasını oku
with open(input_file, "r", encoding="utf-8") as f:
    for line in f:
        record = json.loads(line)
        texts.append(record.get("text") or record.get("content") or "")

# Embedding üretimi
embeddings = []
for text in tqdm(texts, desc="Embedding üretiliyor..."):
    emb = model.encode(text)
    embeddings.append(emb)

embeddings = np.array(embeddings)

# Embedding'leri ve metinleri kaydet
np.save(output_embeddings, embeddings)

with open(output_texts, "w", encoding="utf-8") as f:
    json.dump(texts, f, ensure_ascii=False, indent=2)

print(f"{len(embeddings)} adet embedding üretildi ve kaydedildi:")
print(f"- Embedding dosyası: {output_embeddings}")
print(f"- Metin dosyası: {output_texts}")


In [ ]:
!pip install faiss-cpu numpy

In [ ]:
import json
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# Dosya yolları
JSONL_FILE = r"C:\Users\hukus\Downloads\Code4Her\knowledge_dataset.jsonl"
EMB_PATH = r"C:\Users\hukus\Downloads\Code4Her\knowledge_embeddings.npy"
TEXT_PATH = r"C:\Users\hukus\Downloads\Code4Her\knowledge_texts.json"

# Modeli yükle
model = SentenceTransformer("intfloat/multilingual-e5-base")

texts = []

# JSONL dosyasını oku ve question+answer'i birleştir
with open(JSONL_FILE, "r", encoding="utf-8") as f:
    for line in f:
        record = json.loads(line)
        question = record.get("question", "").strip()
        answer = record.get("answer", "").strip()
        if question and answer:
            # Tek bir metin halinde birleştir
            text = f"Soru: {question} Cevap: {answer}"
            texts.append(text)

print(f"Toplam metin: {len(texts)}")
print("Örnek:", texts[:3])

# Embedding üretimi
embeddings = []
print("Embedding üretiliyor...")
for t in tqdm(texts):
    emb = model.encode(t)
    embeddings.append(emb)

embeddings = np.array(embeddings, dtype="float32")

# Dosyaları kaydet
np.save(EMB_PATH, embeddings)
with open(TEXT_PATH, "w", encoding="utf-8") as f:
    json.dump(texts, f, ensure_ascii=False, indent=2)

print(f"\n{len(texts)} adet embedding üretildi ve kaydedildi:")
print(f"- Embedding dosyası: {EMB_PATH}")
print(f"- Metin dosyası: {TEXT_PATH}")


In [ ]:

import faiss
import numpy as np
import json
from sentence_transformers import SentenceTransformer

# Dosya yolları
EMB_PATH = "/content/knowledge_embeddings.npy"
TEXT_PATH = "/content/knowledge_texts.json"
FAISS_INDEX_PATH = "/content/faiss_index.index"

# Embedding ve metinleri yükle
embeddings = np.load(EMB_PATH)
with open(TEXT_PATH, "r", encoding="utf-8") as f:
    texts = json.load(f)

# FAISS index oluştur
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
faiss.normalize_L2(embeddings)
index.add(embeddings)

# Index'i kaydet
faiss.write_index(index, FAISS_INDEX_PATH)
print(f"FAISS index oluşturuldu ve kaydedildi: {FAISS_INDEX_PATH}")

# Arama fonksiyonu
model = SentenceTransformer("intfloat/multilingual-e5-base")

def search_similar(query, top_k=3):
    query_vec = model.encode([query]).astype("float32")
    faiss.normalize_L2(query_vec)
    D, I = index.search(query_vec, top_k)
    print("\n En benzer kayıtlar:\n")
    for idx, score in zip(I[0], D[0]):
        print(f"Skor: {score:.4f}")
        print(f"Metin: {texts[idx][:500]}...\n")

# Test
search_similar("Sığınağa kimler başvurabilir?")


In [ ]:
!pip uninstall -y langchain langchain-huggingface langchain-community
!pip install -U langchain-huggingface


In [ ]:
!pip install --upgrade langchain faiss-cpu sentence-transformers transformers


In [ ]:
!pip install langchain==0.0.331 sentence-transformers faiss-cpu


In [ ]:
print(index.d)  # FAISS index’in embedding boyutu


In [ ]:
vec = embedding_model.embed_query("test")
print(len(vec))  # embedding boyutu


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "Qwen/Qwen2-0.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7
)

In [ ]:
!pip install -U langchain-community


In [ ]:
!pip install faiss-cpu
!pip install sentence-transformers


In [ ]:
!pip uninstall -y langchain langchain-core langchain-text-splitters langchain-community langchain-classic


In [ ]:
!pip install langchain==1.0.0
!pip install langchain-community==0.4


In [ ]:
!pip install langchain-huggingface


In [ ]:
import faiss
import json
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.docstore import InMemoryDocstore
from langchain.schema.document import Document  # <-- doğru import

# Embedding modeli
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base")

# FAISS index'i yükle
index = faiss.read_index("/content/faiss_index.index")

# Textleri yükle
with open("/content/knowledge_texts.json", "r", encoding="utf-8") as f:
    texts_data = json.load(f)

# Dokümanları Document objesine çevir ve dict oluştur
docs_dict = {str(i): Document(page_content=t) for i, t in enumerate(texts_data)}

# InMemoryDocstore oluştur
docstore = InMemoryDocstore(docs_dict)

# Index ile docstore id eşlemesini oluştur
index_to_docstore_id = list(docs_dict.keys())

# FAISS vectorstore objesi oluştur
vectorstore = FAISS(
    index=index,
    embedding_function=embedding_model,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)


In [1]:
!pip install langchain==1.0.0
!pip install langchain-community==0.4
!pip install langchain-huggingface
!pip install faiss-cpu


In [5]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document  #

print(" Kurulum başarılı! FAISS + LangChain RAG ortamı hazır.")


 Kurulum başarılı! FAISS + LangChain RAG ortamı hazır.


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_name = "Qwen/Qwen2-0.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    repetition_penalty=1.1
)

print(" başarıyla yüklendi!")




/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cpu


 başarıyla yüklendi!


In [7]:
import faiss
import json
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.docstore import InMemoryDocstore
from langchain_core.documents import Document  # ✅ yeni sürümde doğru import

# Embedding modeli
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base")

# FAISS index yükleme
index = faiss.read_index("/content/faiss_index.index")

# Textleri yükle
with open("/content/knowledge_texts.json", "r", encoding="utf-8") as f:
    texts_data = json.load(f)

# Dokümanları Document objesine çevir
docs_dict = {str(i): Document(page_content=t) for i, t in enumerate(texts_data)}

# InMemoryDocstore oluştur
docstore = InMemoryDocstore(docs_dict)

# Index-to-docstore ID listesi
index_to_docstore_id = list(docs_dict.keys())

# FAISS vectorstore oluştur
vectorstore = FAISS(
    index=index,
    embedding_function=embedding_model,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)

print("FAISS vektör veritabanı başarıyla yüklendi")


FAISS vektör veritabanı başarıyla yüklendi


In [8]:
def query_rag(query):

    docs = vectorstore.similarity_search(query, k=3)
    context = "\n".join([d.page_content for d in docs])

    prompt = f"""Kontekst bilgilerini kullanarak soruyu yanıtla.

Kontekst:
{context}

Soru: {query}
Cevap:"""

    result = pipe(prompt)[0]["generated_text"]


    if "Cevap:" in result:
        result = result.split("Cevap:")[-1].strip()

    return result

# Test et
soru = "KADES uygulaması nedir?"
cevap = query_rag(soru)
print("Soru:", soru)
print("\nCevap:", cevap)


Soru: KADES uygulaması nedir?

Cevap: KADES uygulaması, Türkiye'de yaşayan ve tehdit altında olduğunu hisseden veya şiddet görme ihtimali olan tüm kadınlar için kullanıma sunulmuştur. Eşinden, aile üyelerinden veya herhangi bir kişiden şiddet gören ya da tehdit altında olan kadınlar bu uygulamayı kullanabilir. Uygulama, sadece kadın kullanıcılar için geliştirilmiştir. Bu uygulamada, kadınların kendilerine hakları ve özgürlükten etkili olmak için çeşitli politikaları sağlayacaktır. Ayrıca, kendi haklarını korumak ve güvenli bir şekilde yapabilecek hale gelebilir. KADES, şiddet mağdurunun ve genellikle tehdit altında olduğu için hedeflendirdiği gibi, özellikle kadınların kendilerine hakları ve özgürlüğü için uygulanır. Bu uygulamada, kadınların kendilerine hakları ve özgürlüğünü korumak ve güvenli bir şekilde yapabilecek hale gelebilir. Bu uygulamada, kadınların kendilerine hakları ve özgürlükleri ve güvenli bir şekilde korumak için uygulanır. Bu uygulamada, kadınların kendilerine haklar